In [1]:
!unzip 'drive/My Drive/colab_data/cuda_lab/5/data.zip'

Streaming output truncated to the last 5000 lines.
  inflating: data/train/robot/frame79600.jpg  
  inflating: data/train/robot/frame79630.jpg  
  inflating: data/train/robot/frame79750.jpg  
  inflating: data/train/robot/frame79960.jpg  
  inflating: data/train/robot/frame79990.jpg  
  inflating: data/train/robot/frame80020.jpg  
  inflating: data/train/robot/frame80030.jpg  
  inflating: data/train/robot/frame80090.jpg  
  inflating: data/train/robot/frame80260.jpg  
  inflating: data/train/robot/frame80310.jpg  
  inflating: data/train/robot/frame80320.jpg  
  inflating: data/train/robot/frame80340.jpg  
  inflating: data/train/robot/frame80370.jpg  
  inflating: data/train/robot/frame81550.jpg  
  inflating: data/train/robot/frame81610.jpg  
  inflating: data/train/robot/frame81680.jpg  
  inflating: data/train/robot/frame81700.jpg  
  inflating: data/train/robot/frame81770.jpg  
  inflating: data/train/robot/frame81870.jpg  
  inflating: data/train/robot/frame81950.jpg  
  inflati

In [2]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [3]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

avDev = torch.device("cuda:0")

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(avDev)
                labels = labels.to(avDev)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [5]:
model_ft = models.densenet161(pretrained=True) # check: https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(avDev)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/checkpoints/densenet161-8d451a50.pth


In [6]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.0461 Acc: 0.9872
val Loss: 0.0025 Acc: 1.0000

Epoch 1/24
----------
train Loss: 0.0096 Acc: 0.9970
val Loss: 0.0012 Acc: 1.0000

Epoch 2/24
----------
train Loss: 0.0067 Acc: 0.9974
val Loss: 0.0010 Acc: 1.0000

Epoch 3/24
----------
train Loss: 0.0057 Acc: 0.9983
val Loss: 0.0011 Acc: 0.9995

Epoch 4/24
----------
train Loss: 0.0043 Acc: 0.9988
val Loss: 0.0008 Acc: 1.0000

Epoch 5/24
----------
train Loss: 0.0031 Acc: 0.9991
val Loss: 0.0007 Acc: 0.9995

Epoch 6/24
----------
train Loss: 0.0027 Acc: 0.9991
val Loss: 0.0008 Acc: 1.0000

Epoch 7/24
----------
train Loss: 0.0025 Acc: 0.9992
val Loss: 0.0008 Acc: 1.0000

Epoch 8/24
----------
train Loss: 0.0025 Acc: 0.9992
val Loss: 0.0004 Acc: 1.0000

Epoch 9/24
----------
train Loss: 0.0021 Acc: 0.9993
val Loss: 0.0003 Acc: 1.0000

Epoch 10/24
----------
train Loss: 0.0024 Acc: 0.9993
val Loss: 0.0007 Acc: 1.0000

Epoch 11/24
----------
train Loss: 0.0017 Acc: 0.9995
val Loss: 0.0003 Acc: 1.0000

Epoch 12/24
----------
t

In [8]:
torch.save(model_ft.state_dict(), 'drive/My Drive/colab_data/cuda_lab/5/best_model.pth')